In [11]:
import os
from os.path import join
import pandas as pd
import altair as alt
from altair import Chart, X, Y, Text, Axis, TitleParams, datum

In [3]:
def holodata_theme():
    bgColor = '#1E1D1F'
    fgColor = '#ffffff'
    mdColor = '#888888'
    gridColor = '#2F2D32'
    
    return {
        'config': {
            'padding': 20,
            'background': bgColor,
            'view': {
                'stroke': gridColor,
            },
            'title': {
                'color': fgColor,
                'subtitleColor': mdColor,
            },
            'style': {
                'guide-label': {
                    'fill': mdColor,
                },
                'guide-title': {
                    'fill': fgColor,
                    'fontSize': 14
                },
            },
            'axis': {
                'domain': False,
                'grid': True,
                'labels': True,
                'ticks': False,
                'labelPadding': 4,
                'gridColor': gridColor,
                'tickColor': mdColor,
                'titlePadding': 10,
            },
            'text': {
                'color': fgColor
            }
        }
    }


alt.themes.register('holodata', holodata_theme)
alt.themes.enable('holodata')

ThemeRegistry.enable('holodata')

In [4]:
DATASET_DIR = os.environ.get('DATASET_DIR')
!ls $DATASET_DIR

ban_events.csv	   chats_2021-05.csv	  superchats_2021-03.csv
channels.csv	   chats_2021-06.csv	  superchats_2021-04.csv
chats_2021-01.csv  chats_2021-07.csv	  superchats_2021-05.csv
chats_2021-02.csv  chat_stats.csv	  superchats_2021-06.csv
chats_2021-03.csv  dataset-metadata.json  superchats_2021-07.csv
chats_2021-04.csv  deletion_events.csv	  superchat_stats.csv


In [5]:
stats = pd.read_csv(join(DATASET_DIR, 'chat_stats.csv'))
sc_stats = pd.read_csv(join(DATASET_DIR, 'superchat_stats.csv'))
channels = pd.read_csv(join(DATASET_DIR, 'channels.csv'))

# select only active Hololive-affiliated channels
channels = channels[(channels['affiliation'] == 'Hololive') & (channels['group'] != 'INACTIVE')]
channels['group'].fillna('No Group', inplace=True)

# exclude official/secondary/graduated channels
officialChannels = [
    'UCJFZiqLMntJufDCHc6bQixg',
    'UCfrWoRGlawPQDQxxeIDRP0Q',
    'UCotXwY6s8pWmuWd_snKYjhg',
    'UCWsfcksUUpoEvhia0_ut0bA',
]
subChannels = [
    'UCHj_mh57PVMXhAUDphUQDFA',
    'UCLbtM3JZfRTg8v2KGag-RMw',
    'UCp3tgHXw_HI0QMk1K8qh3gQ',
]
graduated = [
    'UCS9uQI-jC3DE0L4IpXyvr6w'
]
channels = channels[~channels['channelId'].isin(officialChannels + subChannels + graduated)]

# merge stats columns
stats_all = pd.merge(stats, sc_stats, on=['channelId', 'period'], how='left')
numeric_columns = stats_all.select_dtypes(include=['number']).columns
stats_all[numeric_columns] = stats_all[numeric_columns].fillna(0).astype(
        'int')
channels = pd.merge(channels, stats_all, on=['channelId'], how='left')

# sex
channels['sex'] = channels['group'].apply(lambda g: 'Male' if g.startswith('Holostars') else 'Female')

# language
def langmatch(channel):
    if channel['group'].startswith('English') or channel['name.en'] == 'IRyS':
        return 'English'
    elif channel['group'].startswith('Indonesia'):
        return 'Indonesian'
    return 'Japanese'
channels['language'] = channels.apply(langmatch, axis=1)

# aggregate data
overall = channels.groupby('name.en').agg({
    'subscriptionCount': 'first',
    'videoCount': 'first',
    'chatCount': 'sum',
    'chatNunique': 'mean',
    'banCount': 'sum',
    'banNunique': 'mean',
    'deletionCount': 'sum',
    'scCount': 'sum',
    'scNunique': 'mean',
    'scTotalJPY': 'sum',
    'scMeanJPY': 'last',
    'affiliation': 'first',
    'group': 'first',
    'name': 'first',
    'sex': 'first',
    'language': 'first',
    'photo': 'first'
}).reset_index()
overall['chatCountPerUser'] = overall['chatCount'] / overall['chatNunique']

In [10]:
# july = overall.sort_values('chatNunique', ascending=False).groupby('group').head(1).reset_index(drop=True)
july = channels.query('period == "2021-07"')

chart = Chart(july,
    title=TitleParams('Live Chat Population (2021/7)', fontSize=25)
).mark_image(
    width=30,
    height=30
).encode(
    X('subscriptionCount',
        title='Subscribers',
        axis=Axis(format='~s')),
    Y('chatNunique',
        title='Unique Chatters',
        axis=Axis(format='~s')),
    url='photo'
).properties(
    width=600,
    height=450
)

# trendline
chart = chart + chart.transform_regression('subscriptionCount', 'chatNunique', method='quad').mark_line(color='#8658FF')

# annotation
chart = chart + Chart(july).mark_text(
    align='left',
    baseline='middle',
    fontSize=20,
    dx=20,
    dy=-5
).encode(
    X('subscriptionCount'),
    Y('chatNunique'),
    Text('chatNunique', format='~s'),
).transform_filter(
    datum['chatNunique'] > 80000
) + Chart(july).mark_text(
    align='left',
    baseline='middle',
    fontSize=10,
    dx=20,
    dy=10
).transform_calculate(subc=f'{datum.subscriptionCount} + " Subs"'
).encode(
    X('subscriptionCount'),
    Y('chatNunique'),
    Text('subscriptionCount', format='~s'),
    # text=Text('subc', type='nominal')
).transform_filter(
    datum['chatNunique'] > 80000
)

# footer
def footer(chart, title, anchor='end'):
    return alt.concat(chart).properties(
        title=TitleParams(
            title,
            anchor=anchor,
            baseline='bottom',
            orient='bottom',
            fontWeight='normal',
            fontSize=10
        )
    )
chart = footer(chart, ['Data Source: Vtuber 300M + Honeybee', 'https://www.kaggle.com/uetchy/vtuber-livechat'])
# chart = footer(chart, 'holodata.org', 'start')

chart


alt.ConcatChart(...)

# User-Channel/Month relation

- authorChannelId
- channelId
- period
- chatCount
- averageChatLength
- scCount
- totalAmountJPY
- averageAmountJPY